In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [3]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
train_df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
import torch

In [5]:
import torchvision
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import DataLoader

In [6]:
model = nn.Sequential()
model.add_module('conv1',nn.Conv2d(in_channels=1,out_channels=32,kernel_size=5,padding=2))
model.add_module('relu1',nn.ReLU())
model.add_module('pool1',nn.MaxPool2d(kernel_size=2))
model.add_module('conv2',nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,padding=2))
model.add_module('relu2',nn.ReLU())
model.add_module('pool2',nn.MaxPool2d(kernel_size=2))

In [7]:
model.add_module('flatten',nn.Flatten())

In [8]:
model.add_module('fc1',nn.Linear(3136,1024))
model.add_module('relu3',nn.ReLU())
model.add_module('dropout', nn.Dropout(p=0.5))
model.add_module('fc2',nn.Linear(1024,10))

In [9]:
x = torch.ones((4,1,28,28))
model(x).shape

torch.Size([4, 10])

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [11]:
model.eval()

Sequential(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3136, out_features=1024, bias=True)
  (relu3): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [12]:
X = train_df.iloc[:, 1:].values / 255.0  
y = train_df.iloc[:, 0].values  
X_test = test_df.values / 255.0
X = X.reshape(-1, 1, 28, 28).astype('float32')
X_test = X_test.reshape(-1, 1, 28, 28).astype('float32')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X, dtype=torch.float32)
y_train_tensor = torch.tensor(y, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [13]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

test_dataset = TensorDataset(X_test_tensor)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [14]:
import torch.optim as optim


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Sequential(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3136, out_features=1024, bias=True)
  (relu3): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [15]:
def train_model(model, train_loader,  criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            
            optimizer.zero_grad()

           
            outputs = model(images)
            loss = criterion(outputs, labels)

            
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
                running_loss = 0.0
        
        
        
        
train_model(model, train_loader, criterion, optimizer, num_epochs=5)

Epoch [1/5], Step [100/657], Loss: 0.5139
Epoch [1/5], Step [200/657], Loss: 0.1480
Epoch [1/5], Step [300/657], Loss: 0.1072
Epoch [1/5], Step [400/657], Loss: 0.0969
Epoch [1/5], Step [500/657], Loss: 0.0806
Epoch [1/5], Step [600/657], Loss: 0.0854
Epoch [2/5], Step [100/657], Loss: 0.0584
Epoch [2/5], Step [200/657], Loss: 0.0606
Epoch [2/5], Step [300/657], Loss: 0.0507
Epoch [2/5], Step [400/657], Loss: 0.0498
Epoch [2/5], Step [500/657], Loss: 0.0413
Epoch [2/5], Step [600/657], Loss: 0.0544
Epoch [3/5], Step [100/657], Loss: 0.0327
Epoch [3/5], Step [200/657], Loss: 0.0356
Epoch [3/5], Step [300/657], Loss: 0.0378
Epoch [3/5], Step [400/657], Loss: 0.0336
Epoch [3/5], Step [500/657], Loss: 0.0436
Epoch [3/5], Step [600/657], Loss: 0.0411
Epoch [4/5], Step [100/657], Loss: 0.0267
Epoch [4/5], Step [200/657], Loss: 0.0251
Epoch [4/5], Step [300/657], Loss: 0.0274
Epoch [4/5], Step [400/657], Loss: 0.0282
Epoch [4/5], Step [500/657], Loss: 0.0271
Epoch [4/5], Step [600/657], Loss:

In [16]:
def predict(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images in test_loader:
            images = images[0].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions


predictions = predict(model, test_loader)
submission2 = pd.DataFrame({'ImageId': list(range(1, len(predictions) + 1)), 'Label': predictions})
submission2.to_csv('submission2.csv', index=False)